In [4]:
import cv2
import mediapipe as mp
import numpy as np

brush_thickness = 15
eraser_thickness = 100

cap = cv2.VideoCapture(0)
if not cap.isOpened():
    print("Error: Couldn't open the camera.")
    exit()

mp_hands = mp.solutions.hands
hands = mp_hands.Hands()
mp_draw = mp.solutions.drawing_utils

xp, yp = 0, 0
img_canvas = np.zeros((720, 1280, 3), np.uint8)
draw_color = (255, 0, 0)  # Initial color (Blue)
drawing = False  # Flag to indicate whether to draw or not

while True:
    success, img = cap.read()
    if not success:
        print("Error: Couldn't read a frame from the camera.")
        break

    img = cv2.flip(img, 1)

    img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    results = hands.process(img_rgb)

    if results.multi_hand_landmarks:
        for hand_landmarks in results.multi_hand_landmarks:
            mp_draw.draw_landmarks(img, hand_landmarks, mp_hands.HAND_CONNECTIONS)
            index_finger = hand_landmarks.landmark[8]
            thumb = hand_landmarks.landmark[4]
            x1, y1 = int(index_finger.x * img.shape[1]), int(index_finger.y * img.shape[0])

            if index_finger.y < hand_landmarks.landmark[6].y and thumb.y < hand_landmarks.landmark[3].y:
                cv2.circle(img, (x1, y1), 15, draw_color, cv2.FILLED)
                if xp == 0 and yp == 0:
                    xp, yp = x1, y1

                if drawing:
                    if draw_color == (255, 255, 255):  # Eraser
                        cv2.line(img_canvas, (xp, yp), (x1, y1), (0, 0, 0), eraser_thickness)  # Set eraser color to black
                    else:
                        cv2.line(img_canvas, (xp, yp), (x1, y1), draw_color, brush_thickness)

                xp, yp = x1, y1

    img_canvas = cv2.resize(img_canvas, (img.shape[1], img.shape[0]))  # Resize img_canvas to match img
    img_gray = cv2.cvtColor(img_canvas, cv2.COLOR_BGR2GRAY)
    _, img_inv = cv2.threshold(img_gray, 50, 255, cv2.THRESH_BINARY_INV)
    img_inv = cv2.cvtColor(img_inv, cv2.COLOR_GRAY2BGR)
    
    img = cv2.bitwise_and(img, img_inv)
    img = cv2.bitwise_or(img, img_canvas)

    cv2.imshow("Virtual Painting", img)
    cv2.imshow("Canvas", img_canvas)

    # Keyboard input to control the program
    key = cv2.waitKey(1)
    if key == 27 or (key == ord('q') and
                    thumb.y < hand_landmarks.landmark[3].y):  # Press 'Esc' or 'q' to exit
        break
    elif key == ord('c'):  # Press 'c' to change color
        draw_color = (np.random.randint(0, 256), np.random.randint(0, 256), np.random.randint(0, 256))
    elif key == ord('e'):  # Press 'e' to use eraser
        draw_color = (255, 255, 255)
        drawing = True
    elif key == ord('p'):  # Press 'p' to pause drawing
        drawing = not drawing

cap.release()
cv2.destroyAllWindows()
